In [19]:
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.utils import to_categorical
import pandas as pd
import numpy as np
import string
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [12]:
# set seeds for reproducability
import tensorflow
tensorflow.random.set_seed(2)

In [4]:
import pandas as pd
import numpy as np
import string, os

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [13]:
# Load headlines data
total_headlines = []
article_df = pd.read_csv('ArticlesApril2017.csv')
total_headlines.extend(list(article_df.headline.values))
total_headlines = [headline for headline in total_headlines if headline != "Unknown"]
len(total_headlines)

831

In [14]:
# Text cleaning function
def clean_data(text):
    text = "".join(character for character in text if character not in string.punctuation).lower()
    text = text.encode("utf8").decode("ascii",'ignore')
    return text

corpus = [clean_data(headline) for headline in total_headlines]
corpus[:10]

['finding an expansive view  of a forgotten people in niger',
 'and now  the dreaded trump curse',
 'venezuelas descent into dictatorship',
 'stain permeates basketball blue blood',
 'taking things for granted',
 'the caged beast awakens',
 'an everunfolding story',
 'oreilly thrives as settlements add up',
 'mouse infestation',
 'divide in gop now threatens trump tax plan']

In [15]:
# Tokenization
tokenizer = Tokenizer()

def get_token_sequences(corpus):
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

input_sequences, total_words = get_token_sequences(corpus)
inp_sequences[:10]

[[169, 17],
 [169, 17, 665],
 [169, 17, 665, 367],
 [169, 17, 665, 367, 4],
 [169, 17, 665, 367, 4, 2],
 [169, 17, 665, 367, 4, 2, 666],
 [169, 17, 665, 367, 4, 2, 666, 170],
 [169, 17, 665, 367, 4, 2, 666, 170, 5],
 [169, 17, 665, 367, 4, 2, 666, 170, 5, 667],
 [6, 80]]

In [16]:
# Padding sequences
def pad_input_sequences(input_sequences):
    max_sequence_length = max([len(seq) for seq in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))
    predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
    label = to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_length

predictors, label, max_sequence_length = pad_input_sequences(input_sequences)

In [17]:
# Model creation
def create_model(max_sequence_length, total_words):
    input_length = max_sequence_length - 1
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_length))
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

model = create_model(max_sequence_length, total_words)
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 18, 10)            24220     
                                                                 
 lstm_1 (LSTM)               (None, 100)               44400     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 2422)              244622    
                                                                 
Total params: 313,242
Trainable params: 313,242
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Model training
model.fit(predictors, label, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

In [20]:
# Text generation function
def generate_text(seed_text, next_words, model, max_sequence_length):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted_probabilities = model.predict(token_list)
        predicted_class_index = np.argmax(predicted_probabilities, axis=1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_class_index:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text.title()

In [24]:
# Generating example headlines
print(generate_text("united states", 5, model, max_sequence_length))
print(generate_text("president trump", 4, model, max_sequence_length))
print(generate_text("donald trump", 4, model, max_sequence_length))
print(generate_text("india and china", 7, model, max_sequence_length))
print(generate_text("new york", 4, model, max_sequence_length))
print(generate_text("science and technology", 30, model, max_sequence_length))

1/1 [==============================] - 0s 16ms/step
United States Turkish Leaders Record As He
1/1 [==============================] - 0s 16ms/step
President Trump With Tough To Vaporize
1/1 [==============================] - 0s 16ms/step
Donald Trump The Wall They Ruin
1/1 [==============================] - 0s 17ms/step
India And China Learned From A Younger Person And The
1/1 [==============================] - 0s 20ms/step
New York Today A Belated Budget
1/1 [==============================] - 0s 16ms/step
Science And Technology Riders A Plan 1930S Plan For Score The Old Days In An Elegy History Tour Heads In Microwave High Grizzled Logos Became Border Own Clues For Days And Black Korea
